In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import utils
from tensorflow.keras import mixed_precision
import os
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
mixed_precision.set_global_policy('mixed_float16')

2024-03-18 20:57:11.087176: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-18 20:57:11.113289: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 20:57:11.113314: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 20:57:11.113340: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-18 20:57:11.119144: I tensorflow/core/platform/cpu_feature_g

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070 Laptop GPU, compute capability 8.9


2024-03-18 20:57:12.939443: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-18 20:57:12.960804: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-18 20:57:12.961007: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
train_data_path = "BIRDS1_split/train"
test_data_path = "BIRDS1_split/test"
IMG_HEIGHT, IMG_WIDTH = 299, 299
BATCH_SIZE = 32
bird_classes = sorted(os.listdir(train_data_path))
bird_classes

['BAY-BREASTED WARBLER',
 'BLUE GROSBEAK',
 'CAPE MAY WARBLER',
 'CERULEAN WARBLER',
 'HEPATIC TANAGER',
 'INDIGO BUNTING',
 'LAZULI BUNTING',
 'OVENBIRD',
 'PYRRHULOXIA',
 'ROSE BREASTED GROSBEAK',
 'SCARLET TANAGER',
 'TOWNSENDS WARBLER',
 'acorn_woodpecker',
 'american_flamingo',
 'american_three_toed_woodpecker',
 'american_wigeon',
 'amethyst_throated_mountain_gem',
 "anna's_hummingbird",
 'baikal_teal',
 'barrows_goldeneye',
 'black_and_white_warbler',
 'black_bellied_whistling_duck',
 'black_chinned_hummingbird',
 'black_headed_grosbeak',
 'black_vulture',
 'blue_jay',
 'blue_winged_warbler',
 'boreal_owl',
 'brant_goose',
 'broad_billed_hummingbird',
 'broad_tailed_hummingbird',
 'buffle_head',
 'california_condor',
 'california_quail',
 'calliope_hummingbird',
 'canada_jay',
 'canvas_back',
 'chukar_partridge',
 'cinnamon_teal',
 'common_goldeneye',
 'costas_hummingbird',
 'dickcissel',
 'downy_woodpecker',
 'emperor_goose',
 'flammulated_owl',
 'fulvous_whistling_duck',
 'gol

In [3]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='training'
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_data_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,
    validation_split=0.2,
    subset='validation'
)

Found 13065 files belonging to 101 classes.
Using 10452 files for training.


2024-03-18 20:57:13.264017: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-18 20:57:13.264228: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-18 20:57:13.264364: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Found 5659 files belonging to 101 classes.
Using 1131 files for validation.


In [4]:
preprocess_input = tf.keras.applications.efficientnet.preprocess_input
train_dataset = train_dataset.map(lambda x, y: (preprocess_input(x), y))
validation_dataset = validation_dataset.map(lambda x, y: (preprocess_input(x), y))

In [5]:
base_model = tf.keras.applications.EfficientNetB0(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)
base_model.trainable = False

for layer in base_model.layers:
    print(layer.name)

input_1
rescaling
normalization
rescaling_1
stem_conv_pad
stem_conv
stem_bn
stem_activation
block1a_dwconv
block1a_bn
block1a_activation
block1a_se_squeeze
block1a_se_reshape
block1a_se_reduce
block1a_se_expand
block1a_se_excite
block1a_project_conv
block1a_project_bn
block2a_expand_conv
block2a_expand_bn
block2a_expand_activation
block2a_dwconv_pad
block2a_dwconv
block2a_bn
block2a_activation
block2a_se_squeeze
block2a_se_reshape
block2a_se_reduce
block2a_se_expand
block2a_se_excite
block2a_project_conv
block2a_project_bn
block2b_expand_conv
block2b_expand_bn
block2b_expand_activation
block2b_dwconv
block2b_bn
block2b_activation
block2b_se_squeeze
block2b_se_reshape
block2b_se_reduce
block2b_se_expand
block2b_se_excite
block2b_project_conv
block2b_project_bn
block2b_drop
block2b_add
block3a_expand_conv
block3a_expand_bn
block3a_expand_activation
block3a_dwconv_pad
block3a_dwconv
block3a_bn
block3a_activation
block3a_se_squeeze
block3a_se_reshape
block3a_se_reduce
block3a_se_expand
blo

In [6]:
inputs = layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3), name="input_layer")

x = base_model(inputs, training=False)
x = layers.GlobalAvgPool2D()(x)

x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)

outputs = layers.Dense(len(bird_classes), activation='softmax', dtype=tf.float32)(x)

model_1_eff_net_b0 = tf.keras.Model(inputs, outputs)

In [7]:
model_1_eff_net_b0.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 299, 299, 3)]     0         
                                                                 
 efficientnetb0 (Functional  (None, 10, 10, 1280)      4049571   
 )                                                               
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 512)               655872    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328

In [8]:
model_1_eff_net_b0.compile(loss='categorical_crossentropy',
                           optimizer=Adam(),
                           metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='Best_models/best_eff_b0.hdf5', verbose=1, save_best_only=True)

history_eff_b0 = model_1_eff_net_b0.fit(train_dataset,
                                        steps_per_epoch=len(train_dataset),
                                        validation_data=validation_dataset,
                                        validation_steps=len(validation_dataset),
                                        epochs=5,
                                        verbose=1,
                                        callbacks=[checkpointer]) 

Epoch 1/5


2024-03-18 20:57:19.152497: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-03-18 20:57:19.209388: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-18 20:57:20.328999: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x71ab300036e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-18 20:57:20.329026: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-03-18 20:57:20.333772: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-18 20:57:20.403929: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


327/327 [==============================] - ETA: 0s - loss: 2.9689 - accuracy: 0.2717
Epoch 1: val_loss improved from inf to 0.98732, saving model to Best_models/best_eff_b0.hdf5
327/327 [==============================] - 26s 65ms/step - loss: 2.9689 - accuracy: 0.2717 - val_loss: 0.9873 - val_accuracy: 0.7657
Epoch 2/5


/home/thefilthysalad/PycharmProjects/BIRD_CLASSIFICATION/.venv/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


326/327 [============================>.] - ETA: 0s - loss: 1.3611 - accuracy: 0.5919
Epoch 2: val_loss improved from 0.98732 to 0.50699, saving model to Best_models/best_eff_b0.hdf5
327/327 [==============================] - 20s 62ms/step - loss: 1.3600 - accuracy: 0.5922 - val_loss: 0.5070 - val_accuracy: 0.8285
Epoch 3/5
326/327 [============================>.] - ETA: 0s - loss: 0.9194 - accuracy: 0.7110
Epoch 3: val_loss improved from 0.50699 to 0.33318, saving model to Best_models/best_eff_b0.hdf5
327/327 [==============================] - 20s 61ms/step - loss: 0.9200 - accuracy: 0.7109 - val_loss: 0.3332 - val_accuracy: 0.9089
Epoch 4/5
326/327 [============================>.] - ETA: 0s - loss: 0.7230 - accuracy: 0.7762
Epoch 4: val_loss improved from 0.33318 to 0.27957, saving model to Best_models/best_eff_b0.hdf5
327/327 [==============================] - 20s 61ms/step - loss: 0.7225 - accuracy: 0.7763 - val_loss: 0.2796 - val_accuracy: 0.9187
Epoch 5/5
326/327 [================

In [9]:
model_1_eff_net_b0.evaluate(validation_dataset)

36/36 [==============================] - 2s 56ms/step - loss: 0.2643 - accuracy: 0.9187


[0.26428791880607605, 0.918656051158905]

In [10]:
for layer in base_model.layers:
    print(layer.name)

len(base_model.layers)

input_1
rescaling
normalization
rescaling_1
stem_conv_pad
stem_conv
stem_bn
stem_activation
block1a_dwconv
block1a_bn
block1a_activation
block1a_se_squeeze
block1a_se_reshape
block1a_se_reduce
block1a_se_expand
block1a_se_excite
block1a_project_conv
block1a_project_bn
block2a_expand_conv
block2a_expand_bn
block2a_expand_activation
block2a_dwconv_pad
block2a_dwconv
block2a_bn
block2a_activation
block2a_se_squeeze
block2a_se_reshape
block2a_se_reduce
block2a_se_expand
block2a_se_excite
block2a_project_conv
block2a_project_bn
block2b_expand_conv
block2b_expand_bn
block2b_expand_activation
block2b_dwconv
block2b_bn
block2b_activation
block2b_se_squeeze
block2b_se_reshape
block2b_se_reduce
block2b_se_expand
block2b_se_excite
block2b_project_conv
block2b_project_bn
block2b_drop
block2b_add
block3a_expand_conv
block3a_expand_bn
block3a_expand_activation
block3a_dwconv_pad
block3a_dwconv
block3a_bn
block3a_activation
block3a_se_squeeze
block3a_se_reshape
block3a_se_reduce
block3a_se_expand
blo

238

In [11]:
base_model.trainable = True
for layer in base_model.layers[:-100]:
    layer.trainable = False

In [12]:
for layer in base_model.layers:
    print(layer.name," :", layer.trainable)

input_1  : False
rescaling  : False
normalization  : False
rescaling_1  : False
stem_conv_pad  : False
stem_conv  : False
stem_bn  : False
stem_activation  : False
block1a_dwconv  : False
block1a_bn  : False
block1a_activation  : False
block1a_se_squeeze  : False
block1a_se_reshape  : False
block1a_se_reduce  : False
block1a_se_expand  : False
block1a_se_excite  : False
block1a_project_conv  : False
block1a_project_bn  : False
block2a_expand_conv  : False
block2a_expand_bn  : False
block2a_expand_activation  : False
block2a_dwconv_pad  : False
block2a_dwconv  : False
block2a_bn  : False
block2a_activation  : False
block2a_se_squeeze  : False
block2a_se_reshape  : False
block2a_se_reduce  : False
block2a_se_expand  : False
block2a_se_excite  : False
block2a_project_conv  : False
block2a_project_bn  : False
block2b_expand_conv  : False
block2b_expand_bn  : False
block2b_expand_activation  : False
block2b_dwconv  : False
block2b_bn  : False
block2b_activation  : False
block2b_se_squeeze  

In [13]:
model_1_eff_net_b0.compile(loss='categorical_crossentropy',
                           optimizer=Adam(),
                           metrics=['accuracy'])
model_1_eff_net_b0.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 299, 299, 3)]     0         
                                                                 
 efficientnetb0 (Functional  (None, 10, 10, 1280)      4049571   
 )                                                               
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 512)               655872    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328

In [14]:
checkpointer = ModelCheckpoint(filepath='Best_models/best_eff_b0.hdf5', verbose=1, save_best_only=True)
history_eff_b0 = model_1_eff_net_b0.fit(train_dataset,
                                        steps_per_epoch=len(train_dataset),
                                        validation_data=validation_dataset,
                                        validation_steps=len(validation_dataset),
                                        epochs=5,
                                        verbose=1,
                                        callbacks=[checkpointer]) 

Epoch 1/5
327/327 [==============================] - ETA: 0s - loss: 1.0680 - accuracy: 0.7182
Epoch 1: val_loss improved from inf to 0.57019, saving model to Best_models/best_eff_b0.hdf5
327/327 [==============================] - 33s 74ms/step - loss: 1.0680 - accuracy: 0.7182 - val_loss: 0.5702 - val_accuracy: 0.8541
Epoch 2/5
326/327 [============================>.] - ETA: 0s - loss: 0.6869 - accuracy: 0.8033
Epoch 2: val_loss improved from 0.57019 to 0.40081, saving model to Best_models/best_eff_b0.hdf5
327/327 [==============================] - 24s 72ms/step - loss: 0.6881 - accuracy: 0.8030 - val_loss: 0.4008 - val_accuracy: 0.8674
Epoch 3/5
326/327 [============================>.] - ETA: 0s - loss: 0.5796 - accuracy: 0.8359
Epoch 3: val_loss improved from 0.40081 to 0.37916, saving model to Best_models/best_eff_b0.hdf5
327/327 [==============================] - 23s 69ms/step - loss: 0.5800 - accuracy: 0.8358 - val_loss: 0.3792 - val_accuracy: 0.8930
Epoch 4/5
326/327 [==========